In [1]:

import pandas as pd
import numpy as np

data = pd.read_csv("./DATA/training.csv", encoding='ISO8859-2')
test = pd.read_csv("./DATA/test.csv", encoding='ISO8859-2')
data.head(5).style

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"More I can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of types of accidents -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know. There appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


In [3]:
data_all = pd.concat([data, test])

In [4]:
y_columns = data.columns[2:]
y_columns

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [10]:
def remove_strip(str):
    return str.replace("\r", "").replace("\n", "").strip()

def get_stop_words(f, encoding='utf-8'):
    stop_words = []
    with open(f, "r", encoding=encoding) as f_stopwords:
        for line in f_stopwords:
            line = remove_strip(line)
            stop_words.append(line.lower())
    stop_words = set(stop_words)
    print("total counts: ", len(stop_words))

    return stop_words

def read_corpus(fname, tokens_only=False, encoding='utf-8'):
    import gensim
    import smart_open
    with smart_open.open(fname, encoding=encoding) as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])
stop_words = get_stop_words("stopwords.txt")
data_all['comment_text_clean'] = data_all['comment_text'].apply(
    lambda text: " ".join([w for w in text.replace(",", "").replace("?", "").replace(".", "").replace("!", "").split() if w.lower() not in stop_words]))
data_all['comment_text_clean'].to_csv("./DATA/lines.csv")
train_corpus = list(read_corpus("./DATA/lines.csv"))

total counts:  233


In [11]:
from gensim.models import  doc2vec
model = doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [14]:
model.save(fname_or_handle="./DATA/doc2vec")

In [28]:
comment_vec = data_all['comment_text_clean'].apply(lambda doc: model.infer_vector(doc.split()))
data_comment = pd.DataFrame(comment_vec.tolist())
data_comment

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.659544,0.355920,-0.357711,0.169250,-0.137454,0.272444,0.369584,-0.511792,0.209280,0.325373,...,-0.066734,0.162196,-0.295918,0.119517,0.256937,0.158404,-0.104761,0.233320,0.567666,-0.101431
1,0.260232,0.141744,-0.076490,0.158042,0.116818,0.194678,-0.097758,-0.337226,0.151364,0.227388,...,-0.061192,-0.280899,-0.055993,-0.114491,0.205421,0.018913,-0.207736,0.202338,-0.055172,-0.057435
2,-0.190541,-0.060816,-0.540582,0.633664,-0.349069,-0.926886,-0.349358,-0.287507,-0.338681,0.445484,...,-0.706665,-0.552445,0.120478,0.131730,0.358549,0.319914,0.365362,-0.249613,-0.243781,-0.305363
3,1.104135,-0.493525,-0.628478,-0.681581,0.592711,-0.299966,0.104307,-0.103474,-0.233336,0.419988,...,-0.074779,0.538066,-0.986933,-0.081675,0.398282,-0.347575,-0.877894,-0.409141,-0.855466,-0.581544
4,0.076409,-0.001454,0.089777,-0.047148,-0.230956,0.043232,-0.001521,-0.182370,-0.359984,-0.225730,...,-0.333531,-0.234021,0.202028,0.075737,0.163032,0.160580,-0.063708,0.089321,-0.370965,-0.075388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159566,0.250824,0.092205,-0.027755,0.151624,0.064720,-0.464031,-0.042186,-0.012589,0.343600,0.384450,...,-0.719529,0.175069,-0.095379,-0.503314,0.113950,-0.078214,0.047274,-0.003633,-0.284261,-0.175465
159567,0.227809,0.192934,0.015739,0.118516,-0.022189,-0.086921,-0.021833,0.024604,0.066078,0.226871,...,-0.101284,-0.047199,0.138576,-0.113423,0.099395,-0.135330,-0.012029,-0.070452,-0.140136,-0.158165
159568,-0.000501,-0.159231,0.928774,-0.291422,0.820592,0.902826,-2.400964,-1.088227,-0.408307,0.255292,...,-1.268946,0.563799,-0.041278,-1.512789,1.982134,0.085144,-0.398849,-0.760662,-0.253909,-1.728271
159569,0.543212,0.041381,-0.469375,0.223098,-0.427424,0.064037,-0.230691,-0.242991,0.626162,0.225249,...,-0.009033,0.209465,0.150699,0.113015,0.220070,0.244850,-0.419988,-0.402074,0.663770,-0.758954


In [29]:
data_comment.to_csv("./DATA/comment_doc2vec.csv", index=None)

In [ ]:
data

In [31]:
data_doc2vec = data_comment.loc[:127656,]
test_doc2vec = data_comment.loc[127657:,]

In [32]:
random_seed = 42
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score

models = []
i = 0
test_size = 0.25
rus = RandomUnderSampler(random_state=random_seed, )
X, y= rus.fit_resample(data_doc2vec, data[y_columns[i]])
X.shape

(24364, 300)

In [34]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=random_seed)
lgb =LGBMClassifier(max_depth=8, num_leaves=200, boosting_type='dart', random_state=random_seed)
lgb.fit(X_train, y_train)
y_train_pred = lgb.predict(X_train)
y_valid_pred = lgb.predict(X_valid)
print("train set f1 score: ", f1_score(y_train, y_train_pred))
print("training set %s f1_score: %.2f" % (y_columns[i], f1_score(y_valid, y_valid_pred) * 100))
models.append(lgb)

train set f1 score:  0.9697877118870648
training set toxic f1_score: 76.98
